In [1]:
import numpy as np

In [9]:
hid = 768
inter = 3072
num_exps = 4
num_loras = 16
lora_rank = 16
lora_top_k = 2


In [10]:
hetro_flops = 2 * lora_top_k * ((hid * lora_rank) + (inter * lora_rank))
hetro_params = 2 * num_loras * ((hid * lora_rank) + (inter * lora_rank))

In [11]:
expert_params = hid * inter * 2
expert_flops = hid * inter * 2

moe_expert_params = 4 * expert_params
moe_expert_flops = expert_params

minion_expert_params = 3 * expert_params + hetro_params
minion_expert_flops =  (0.75 * moe_expert_flops) + (0.25 * hetro_flops)


In [12]:
((moe_expert_flops - minion_expert_flops) / moe_expert_flops) * 100

23.697916666666664

In [13]:
minion_expert_params

16121856

In [14]:
minion_expert_flops

3600384.0

In [8]:
(((768 * 2624) * 2) - minion_expert_flops)/ ((768 * 2624) * 2) 

0.10670731707317073

In [16]:
((768 * 2624) * 2) * 4

16121856